# 7 합성곱 신경망


In [2]:
# 7.4 4차원 배열
import numpy as np


x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [5]:
print(x[0].shape) # 1. 28. 28
print(x[1].shape) # 1. 28. 28


(1, 28, 28)
(1, 28, 28)
[[6.34383921e-01 3.56736032e-01 2.53743849e-02 5.03395407e-01
  9.91949785e-01 6.71400075e-01 7.91469611e-02 4.38790921e-01
  5.94103476e-01 3.72575837e-02 2.84112648e-01 3.41816994e-01
  8.27318794e-01 8.40057857e-01 8.32662569e-01 7.74896955e-01
  7.51337247e-01 9.05780253e-01 6.25963064e-01 4.26240671e-01
  9.16588542e-01 5.52798788e-01 7.30118651e-01 7.63582424e-01
  9.25995717e-01 1.90757369e-01 8.08994563e-01 2.00556258e-01]
 [8.28323523e-01 1.78317100e-01 1.11237541e-02 7.33647478e-01
  7.57228725e-01 7.57368997e-01 8.85896547e-01 9.63296298e-01
  4.71810309e-01 5.63974676e-02 8.10845099e-01 2.92343160e-01
  3.65235257e-01 1.34140274e-01 7.04840358e-01 4.14711016e-01
  5.27189957e-01 4.08830878e-01 7.85636002e-01 1.24629254e-01
  4.81392104e-01 3.11126250e-01 6.33201077e-01 1.83668271e-01
  7.19244726e-01 4.14237123e-01 3.73697269e-01 9.30690189e-01]
 [2.54376852e-01 4.91319248e-01 3.33101243e-01 3.14520950e-01
  2.73647777e-01 4.15913764e-01 7.32926016e-

In [5]:
# 7_4_3 합성곱 계층 구현하기
import numpy as np
import sys, os
sys.path.append(os.pardir)
from util import im2col

x1 = np.random.rand(1,3,7,7) # (데이터 수, 채널 수 , 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0) # 스트라이드 1
print(col1.shape) # 9,75

x2 = np.random.rand(10,3,7,7) # 데이터 10개로 변경
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # 90 75



(9, 75)
(90, 75)


In [7]:
class Convolution:
    def __init__(self, W, b, stride =1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        
        out_h = int(1+(H+2*self.pad-FH)/self.stride)
        out_w = int(1+(W+2*self.pad-PW)/self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out
    

In [9]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad =0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+(H-self.pool_h)/self.stride)
        out_w = int(1+(W-self.pool_w)/self.stride)
        
        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # 최댓값(2)
        out = np.max(col, axis=1) # 가로 전개
        
        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out
    

In [12]:
# 5. CNN 구현

class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28),
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0,'stride':1},
                hidden_size=100, output_size = 10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/\
                filter_stride +1
        pool_output_size = int(filter_num * (conv_output_size/2)*(conv_output_size/2))
        
        # 가중치 매개변수 초기화
        self.params ={}
        self.params['W1'] = weight_init_std * \
                        np.random.rand(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                        np.random.rand(pool_output_size, hidden_size)
        
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                        np.random.rand(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN 구성 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                          conv_param['stride'],conv_param['pad'])
        
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h = 2, pool_w = 2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'].self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    # 추론을 수행하는 predit 메서드
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    # 손실 함수를 구하는 loss
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    # x는 입력 데이터 t는 정답 레이블
    
    
    # 오차역전파법 기울기 구하는 메서드
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads